In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot


#créer une session dans le master
spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-readAll").getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-readAll").getOrCreate()
        
#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdc-httpswww/**")

readall.take(5)

import pyspark.sql.functions as f
csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csall.createOrReplaceTempView("CSET")
csall.show(50,truncate=100)

In [ ]:
def upsetplotofclass(cname):
    persons=spark.sql("select pset,count from CSET where pset like '%isa:<schema.org/"+cname+">%' ")
    persons = persons.withColumn("pset", split(persons["pset"], " "))
    persons.createOrReplaceTempView("persons")
    top_persons= spark.sql("""
    SELECT  *
    FROM persons
    limit 20
    """)
    pandas_persons = top_persons.toPandas()

    # Create a list of sets for each row in the DataFrame
    pset_list = [set(x) for x in pandas_persons['pset']]


    # Create the UpSetplot
    upset_data = upsetplot.from_memberships(pset_list, data=pandas_persons['count'])
    upsetplot.plot(upset_data, show_counts=True)
    plt.show()
    

In [ ]:
upsetplotofclass("Recipe")